<a href="https://colab.research.google.com/github/dlgur1994/Hackerthon1_Aiffel/blob/main/Recruit_Restaurant_Visitor_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 사이트의 예약, 방문 및 기타 정보를 사용하여 지정된 날짜에 미래의 레스토랑 방문자 합계를 예측해야 합니다. 교육 데이터는 <u>2016년부터 2017년 4월까지</u>를 포함합니다.        

테스트 세트는 <u>2017년 4월과 5월의 마지막 주</u>에 걸쳐 있습니다. 


테스트 세트는 시간(선착순, 일반에 이은 개인 폴더)에 따라 분할되며 선택된 일부 에어 레스토랑에 적용됩니다. 테스트 세트는 <u>의도적으로 "골든 위크"라고 불리는 일본의 휴일 주에 걸쳐 진행</u>됩니다."

시험장에서 식당이 문을 닫고 손님이 없는 날이 있다. 점수 매기기에서는 무시됩니다. 그 훈련 세트에는 식당들이 문을 닫은 날들이 생략되어 있다.

In [88]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import matplotlib.pyplot as plt
from IPython.display import display
pd.set_option('display.float_format', lambda x: '%.5f' % x)


In [89]:
air_reserve_df = pd.read_csv('/content/drive/MyDrive/Hackathon/recruit-restaurant-visitor-forecasting/air_reserve.csv.zip')
air_store_info_df = pd.read_csv('/content/drive/MyDrive/Hackathon/recruit-restaurant-visitor-forecasting/air_store_info.csv.zip')
hpg_reserve_df = pd.read_csv('/content/drive/MyDrive/Hackathon/recruit-restaurant-visitor-forecasting/hpg_reserve.csv.zip')
hpg_store_info_df = pd.read_csv('/content/drive/MyDrive/Hackathon/recruit-restaurant-visitor-forecasting/hpg_store_info.csv.zip')
air_visit_data_df = pd.read_csv('/content/drive/MyDrive/Hackathon/recruit-restaurant-visitor-forecasting/air_visit_data.csv.zip')
store_id_relation_df = pd.read_csv('/content/drive/MyDrive/Hackathon/recruit-restaurant-visitor-forecasting/store_id_relation.csv.zip')
date_info_df = pd.read_csv('/content/drive/MyDrive/Hackathon/recruit-restaurant-visitor-forecasting/date_info.csv.zip')

In [90]:
display(air_res.head())
air_res.info()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92378 entries, 0 to 92377
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   air_store_id      92378 non-null  object
 1   visit_datetime    92378 non-null  object
 2   reserve_datetime  92378 non-null  object
 3   reserve_visitors  92378 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 2.8+ MB


In [91]:
display(air_store.head())
air_store.info()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.69512,135.19785
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.69512,135.19785
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.69512,135.19785
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.69512,135.19785
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.65807,139.75160


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   air_store_id    829 non-null    object 
 1   air_genre_name  829 non-null    object 
 2   air_area_name   829 non-null    object 
 3   latitude        829 non-null    float64
 4   longitude       829 non-null    float64
dtypes: float64(2), object(3)
memory usage: 32.5+ KB


In [92]:
display(air_visit.head())
display(air_visit.info())
hpg_res.info()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252108 entries, 0 to 252107
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   air_store_id  252108 non-null  object
 1   visit_date    252108 non-null  object
 2   visitors      252108 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 5.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000320 entries, 0 to 2000319
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   hpg_store_id      object
 1   visit_datetime    object
 2   reserve_datetime  object
 3   reserve_visitors  int64 
dtypes: int64(1), object(3)
memory usage: 61.0+ MB


In [93]:
id_rel.head()
id_rel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   air_store_id  150 non-null    object
 1   hpg_store_id  150 non-null    object
dtypes: object(2)
memory usage: 2.5+ KB


In [94]:
air_res.air_store_id.nunique() # nujmber of unique restaurants in air system

314

In [95]:
hpg_res.hpg_store_id.nunique()

13325

In [96]:
id_rel.air_store_id.nunique()

150

## Data Merging
: 데이터 전부를 합처 하나의 데이터를 만든다

In [97]:
# air 예약정보와 가게정보 합치기
air_mixed_df = pd.merge(air_reserve_df, air_store_info_df)
hpg_mixed_df = pd.merge(hpg_reserve_df, hpg_store_info_df)
hpg_mixed_df.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,French,Hyōgo-ken Kōbe-shi None,34.69211,135.19170
1,hpg_dac72789163a3f47,2016-01-02 12:00:00,2016-01-01 20:00:00,2,French,Hyōgo-ken Kōbe-shi None,34.69211,135.19170
2,hpg_dac72789163a3f47,2016-01-03 19:00:00,2016-01-02 15:00:00,2,French,Hyōgo-ken Kōbe-shi None,34.69211,135.19170
3,hpg_dac72789163a3f47,2016-01-06 12:00:00,2016-01-06 08:00:00,2,French,Hyōgo-ken Kōbe-shi None,34.69211,135.19170
4,hpg_dac72789163a3f47,2016-01-10 17:00:00,2016-01-04 22:00:00,3,French,Hyōgo-ken Kōbe-shi None,34.69211,135.19170


In [98]:
# column name 공통되게 변경
air_mixed_new_columns_df = air_mixed_df.rename(columns = {"air_store_id":"id", "air_genre_name":"genre", "air_area_name": "area_name"}, inplace = False)
hpg_mixed_new_columns_df = hpg_mixed_df.rename(columns = {"hpg_store_id":"id", "hpg_genre_name":"genre", "hpg_area_name": "area_name"}, inplace = False)
mixed_df = pd.concat([air_mixed_new_columns_df, hpg_mixed_new_columns_df])
mixed_df

,id,visit_datetime,reserve_datetime,reserve_visitors,genre,area_name,latitude,longitude
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.65807,139.75160
1,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.65807,139.75160
2,air_877f79706adbfb06,2016-01-02 18:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.65807,139.75160
3,air_877f79706adbfb06,2016-01-02 21:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.65807,139.75160
4,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-04 20:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.65807,139.75160
...,...,...,...,...,...,...,...,...
871303,hpg_e106ddef2b137959,2017-04-24 18:00:00,2017-04-20 14:00:00,8,Japanese style,Tōkyō-to Chūō-ku None,35.68250,139.77362
871304,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-22 14:00:00,14,Japanese style,Tōkyō-to Chūō-ku None,35.68250,139.77362
871305,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-22 14:00:00,15,Japanese style,Tōkyō-to Chūō-ku None,35.68250,139.77362
871306,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-10 17:00:00,18,Japanese style,Tōkyō-to Chūō-ku None,35.68250,139.77362


In [99]:
# visit_datetime을 년월일로 변경
mixed_cp_df = mixed_df.copy()
mixed_cp_df['visit_date'] = mixed_cp_df['visit_datetime'].str.slice(start=0, stop=10)
# mixed_cp_df= mixed_cp_df.drop(['id', 'visit_datetime', 'reserve_datetime'], axis='columns')
mixed_cp_df

,id,visit_datetime,reserve_datetime,reserve_visitors,genre,area_name,latitude,longitude,visit_date
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.65807,139.75160,2016-01-01
1,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.65807,139.75160,2016-01-01
2,air_877f79706adbfb06,2016-01-02 18:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.65807,139.75160,2016-01-02
3,air_877f79706adbfb06,2016-01-02 21:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.65807,139.75160,2016-01-02
4,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-04 20:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.65807,139.75160,2016-01-08
...,...,...,...,...,...,...,...,...,...
871303,hpg_e106ddef2b137959,2017-04-24 18:00:00,2017-04-20 14:00:00,8,Japanese style,Tōkyō-to Chūō-ku None,35.68250,139.77362,2017-04-24
871304,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-22 14:00:00,14,Japanese style,Tōkyō-to Chūō-ku None,35.68250,139.77362,2017-04-24
871305,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-22 14:00:00,15,Japanese style,Tōkyō-to Chūō-ku None,35.68250,139.77362,2017-04-24
871306,hpg_e106ddef2b137959,2017-04-24 19:00:00,2017-04-10 17:00:00,18,Japanese style,Tōkyō-to Chūō-ku None,35.68250,139.77362,2017-04-24


In [100]:
# 같은 날, 같은 식당의 reserve_visitors 합치기
merged_df = mixed_cp_df.groupby(by=['visit_date','latitude','longitude','genre','area_name', 'id'], as_index=False).sum()
merged_df

,visit_date,latitude,longitude,genre,area_name,id,reserve_visitors
0,2016-01-01,33.55688,130.41513,Seafood,Fukuoka-ken Fukuoka-shi None,hpg_a508a4e8b427279a,6
1,2016-01-01,33.57638,130.33931,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_b7f05c204af42b91,6
2,2016-01-01,33.57638,130.33931,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_f802d9079d5b1af3,2
3,2016-01-01,33.57973,130.40176,International cuisine,Fukuoka-ken Fukuoka-shi Ōmiya,hpg_962b3ed9f41e5b63,2
4,2016-01-01,33.58697,130.39280,Grilled meat,Fukuoka-ken Fukuoka-shi Daimyō,hpg_f6117f6d2533237f,2
...,...,...,...,...,...,...,...
590934,2017-05-31,35.68250,139.77362,Seafood,Tōkyō-to Chūō-ku None,hpg_77d300f2b1f22a8a,8
590935,2017-05-31,35.69138,139.70126,Spain Bar/Italian Bar,Tōkyō-to Shinjuku-ku None,hpg_ba19e6108767ebb6,4
590936,2017-05-31,35.69138,139.70126,Steak/Hamburger/Curry,Tōkyō-to Shinjuku-ku None,hpg_2bc4e1cbe5cd308b,1
590937,2017-05-31,35.69578,139.76845,International cuisine,Tōkyō-to Chiyoda-ku None,hpg_24bce7c3238bdd5c,40


In [101]:
# 요일과 holiday flag 추가하기
merged_with_day_df = pd.merge(merged_df, date_info_df, left_on='visit_date', right_on='calendar_date')
merged_with_day_df

,visit_date,latitude,longitude,genre,area_name,id,reserve_visitors,calendar_date,day_of_week,holiday_flg
0,2016-01-01,33.55688,130.41513,Seafood,Fukuoka-ken Fukuoka-shi None,hpg_a508a4e8b427279a,6,2016-01-01,Friday,1
1,2016-01-01,33.57638,130.33931,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_b7f05c204af42b91,6,2016-01-01,Friday,1
2,2016-01-01,33.57638,130.33931,Japanese style,Fukuoka-ken Fukuoka-shi Minamishō,hpg_f802d9079d5b1af3,2,2016-01-01,Friday,1
3,2016-01-01,33.57973,130.40176,International cuisine,Fukuoka-ken Fukuoka-shi Ōmiya,hpg_962b3ed9f41e5b63,2,2016-01-01,Friday,1
4,2016-01-01,33.58697,130.39280,Grilled meat,Fukuoka-ken Fukuoka-shi Daimyō,hpg_f6117f6d2533237f,2,2016-01-01,Friday,1
...,...,...,...,...,...,...,...,...,...,...
590934,2017-05-31,35.68250,139.77362,Seafood,Tōkyō-to Chūō-ku None,hpg_77d300f2b1f22a8a,8,2017-05-31,Wednesday,0
590935,2017-05-31,35.69138,139.70126,Spain Bar/Italian Bar,Tōkyō-to Shinjuku-ku None,hpg_ba19e6108767ebb6,4,2017-05-31,Wednesday,0
590936,2017-05-31,35.69138,139.70126,Steak/Hamburger/Curry,Tōkyō-to Shinjuku-ku None,hpg_2bc4e1cbe5cd308b,1,2017-05-31,Wednesday,0
590937,2017-05-31,35.69578,139.76845,International cuisine,Tōkyō-to Chiyoda-ku None,hpg_24bce7c3238bdd5c,40,2017-05-31,Wednesday,0


In [102]:
# air_visit_data 이용해서 워크인, 노쇼 등 확인
check_no_show_df = pd.merge(air_visit_data_df, merged_with_day_df, left_on=['air_store_id', 'visit_date'], right_on=['id','visit_date'])
check_no_show2_df = check_no_show_df.drop(['id', 'latitude', 'longitude', 'genre', 'area_name', 'calendar_date', 'day_of_week', 'holiday_flg'], axis='columns')
check_no_show2_df

,air_store_id,visit_date,visitors,reserve_visitors
0,air_35512c42db0868da,2016-02-27,6,12
1,air_ee3a01f0c71a769f,2016-01-04,61,2
2,air_ee3a01f0c71a769f,2016-01-08,21,3
3,air_ee3a01f0c71a769f,2016-01-09,57,25
4,air_ee3a01f0c71a769f,2016-01-10,32,5
...,...,...,...,...
28059,air_754ae581ad80cc9f,2017-04-14,3,2
28060,air_754ae581ad80cc9f,2017-04-15,39,20
28061,air_754ae581ad80cc9f,2017-04-19,6,6
28062,air_754ae581ad80cc9f,2017-04-22,47,28


In [103]:
# 노쇼와 워크인의 비중이 큰듯하여 예약에 관한 내용을 신경쓰지 않기로 함
temp_df = check_no_show_df.drop(['id', 'calendar_date'], axis='columns')
temp_df

,air_store_id,visit_date,visitors,latitude,longitude,genre,area_name,reserve_visitors,day_of_week,holiday_flg
0,air_35512c42db0868da,2016-02-27,6,35.71778,139.56626,Dining bar,Tōkyō-to Musashino-shi Midorichō,12,Saturday,0
1,air_ee3a01f0c71a769f,2016-01-04,61,34.71090,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,2,Monday,0
2,air_ee3a01f0c71a769f,2016-01-08,21,34.71090,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,3,Friday,0
3,air_ee3a01f0c71a769f,2016-01-09,57,34.71090,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,25,Saturday,0
4,air_ee3a01f0c71a769f,2016-01-10,32,34.71090,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,5,Sunday,0
...,...,...,...,...,...,...,...,...,...,...
28059,air_754ae581ad80cc9f,2017-04-14,3,37.87422,138.97194,Izakaya,Niigata-ken Niigata-shi Teraohigashi,2,Friday,0
28060,air_754ae581ad80cc9f,2017-04-15,39,37.87422,138.97194,Izakaya,Niigata-ken Niigata-shi Teraohigashi,20,Saturday,0
28061,air_754ae581ad80cc9f,2017-04-19,6,37.87422,138.97194,Izakaya,Niigata-ken Niigata-shi Teraohigashi,6,Wednesday,0
28062,air_754ae581ad80cc9f,2017-04-22,47,37.87422,138.97194,Izakaya,Niigata-ken Niigata-shi Teraohigashi,28,Saturday,0


In [104]:
# area_name none 확인
# temp_df['area_name'].unique()
temp_df['area_name'].value_counts(ascending=True)

Tōkyō-to Taitō-ku Asakusa                        1
Hyōgo-ken Kōbe-shi Motomachidōri                 1
Tōkyō-to Musashino-shi Midorichō                 1
Tōkyō-to Setagaya-ku Okusawa                     1
Hyōgo-ken Takarazuka-shi Tōyōchō                 1
                                              ... 
Shizuoka-ken Hamamatsu-shi Motoshirochō       1349
Hokkaidō Sapporo-shi Minami 3 Jōnishi         1421
Hiroshima-ken Hiroshima-shi Kokutaijimachi    1710
Fukuoka-ken Fukuoka-shi Daimyō                1801
Tōkyō-to Shibuya-ku Shibuya                   2158
Name: area_name, Length: 71, dtype: int64

In [105]:
# genre 값 확인
# temp_df['genre'].unique()
temp_df['genre'].value_counts(ascending=True)

Asian                              1
International cuisine              3
Karaoke/Party                     43
Other                            248
Creative cuisine                 331
Bar/Cocktail                     426
Okonomiyaki/Monja/Teppanyaki     932
Western food                     950
Cafe/Sweets                     1290
Yakiniku/Korean food            1508
Japanese food                   3076
Dining bar                      3406
Italian/French                  6071
Izakaya                         9779
Name: genre, dtype: int64

In [106]:
# genre 통합
final_df = temp_df.copy()
final_df['genre'] = final_df['genre'].replace('Asian', 'Other')
final_df['genre'] = final_df['genre'].replace('International cuisine', 'Other')
# final_df['genre'].value_counts(ascending=True)
final_df

,air_store_id,visit_date,visitors,latitude,longitude,genre,area_name,reserve_visitors,day_of_week,holiday_flg
0,air_35512c42db0868da,2016-02-27,6,35.71778,139.56626,Dining bar,Tōkyō-to Musashino-shi Midorichō,12,Saturday,0
1,air_ee3a01f0c71a769f,2016-01-04,61,34.71090,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,2,Monday,0
2,air_ee3a01f0c71a769f,2016-01-08,21,34.71090,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,3,Friday,0
3,air_ee3a01f0c71a769f,2016-01-09,57,34.71090,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,25,Saturday,0
4,air_ee3a01f0c71a769f,2016-01-10,32,34.71090,137.72594,Cafe/Sweets,Shizuoka-ken Hamamatsu-shi Motoshirochō,5,Sunday,0
...,...,...,...,...,...,...,...,...,...,...
28059,air_754ae581ad80cc9f,2017-04-14,3,37.87422,138.97194,Izakaya,Niigata-ken Niigata-shi Teraohigashi,2,Friday,0
28060,air_754ae581ad80cc9f,2017-04-15,39,37.87422,138.97194,Izakaya,Niigata-ken Niigata-shi Teraohigashi,20,Saturday,0
28061,air_754ae581ad80cc9f,2017-04-19,6,37.87422,138.97194,Izakaya,Niigata-ken Niigata-shi Teraohigashi,6,Wednesday,0
28062,air_754ae581ad80cc9f,2017-04-22,47,37.87422,138.97194,Izakaya,Niigata-ken Niigata-shi Teraohigashi,28,Saturday,0


## 레이블 인코딩
---

In [194]:
from sklearn.preprocessing import LabelEncoder

final_encode_df = final_df.copy()

encoder = LabelEncoder()
encode_label = encoder.fit_transform(final_encode_df['area_name'].values)
final_encode_df['area_name'] = encode_label
print(encode_label)
print(encoder.inverse_transform(encode_label))

encode_label = encoder.fit_transform(final_encode_df['genre'].values)
final_encode_df['genre'] = encode_label
print(encode_label)
print(encoder.inverse_transform(encode_label))

map_dayofweek = {'Sunday':0, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6}
final_encode_df[['day_of_week']] = final_encode_df[['day_of_week']].applymap(map_dayofweek.get)

final_encode_df

[46 29 29 ... 28 28 39]
['Tōkyō-to Musashino-shi Midorichō'
 'Shizuoka-ken Hamamatsu-shi Motoshirochō'
 'Shizuoka-ken Hamamatsu-shi Motoshirochō' ...
 'Niigata-ken Niigata-shi Teraohigashi'
 'Niigata-ken Niigata-shi Teraohigashi' 'Tōkyō-to Kōtō-ku Tomioka']
[False]
[3 1 1 ... 5 5 5]
['Dining bar' 'Cafe/Sweets' 'Cafe/Sweets' ... 'Izakaya' 'Izakaya'
 'Izakaya']


,air_store_id,visit_date,visitors,latitude,longitude,genre,area_name,reserve_visitors,day_of_week,holiday_flg
0,air_35512c42db0868da,2016-02-27,6,35.71778,139.56626,3,46,12,6,0
1,air_ee3a01f0c71a769f,2016-01-04,61,34.71090,137.72594,1,29,2,1,0
2,air_ee3a01f0c71a769f,2016-01-08,21,34.71090,137.72594,1,29,3,5,0
3,air_ee3a01f0c71a769f,2016-01-09,57,34.71090,137.72594,1,29,25,6,0
4,air_ee3a01f0c71a769f,2016-01-10,32,34.71090,137.72594,1,29,5,0,0
...,...,...,...,...,...,...,...,...,...,...
28059,air_754ae581ad80cc9f,2017-04-14,3,37.87422,138.97194,5,28,2,5,0
28060,air_754ae581ad80cc9f,2017-04-15,39,37.87422,138.97194,5,28,20,6,0
28061,air_754ae581ad80cc9f,2017-04-19,6,37.87422,138.97194,5,28,6,3,0
28062,air_754ae581ad80cc9f,2017-04-22,47,37.87422,138.97194,5,28,28,6,0


## 원핫 인코딩

In [212]:
final_onehotencode_df = final_df.copy()

final_onehotencode_df = pd.get_dummies(final_onehotencode_df, columns = ['genre'])
final_onehotencode_df = pd.get_dummies(final_onehotencode_df, columns = ['area_name'])
final_onehotencode_df = pd.get_dummies(final_onehotencode_df, columns = ['day_of_week'])

final_onehotencode_df

,air_store_id,visit_date,visitors,latitude,longitude,reserve_visitors,holiday_flg,genre_Bar/Cocktail,genre_Cafe/Sweets,genre_Creative cuisine,genre_Dining bar,genre_Italian/French,genre_Izakaya,genre_Japanese food,genre_Karaoke/Party,genre_Okonomiyaki/Monja/Teppanyaki,genre_Other,genre_Western food,genre_Yakiniku/Korean food,area_name_Fukuoka-ken Fukuoka-shi Daimyō,area_name_Fukuoka-ken Fukuoka-shi Hakata Ekimae,area_name_Fukuoka-ken Fukuoka-shi Momochi,area_name_Fukuoka-ken Fukuoka-shi Shiobaru,area_name_Fukuoka-ken Fukuoka-shi Takatori,area_name_Fukuoka-ken Kitakyūshū-shi None,area_name_Fukuoka-ken Kitakyūshū-shi Ōtemachi,area_name_Fukuoka-ken Kurume-shi Jōnanmachi,area_name_Fukuoka-ken Yame-shi Motomachi,area_name_Hiroshima-ken Fukuyama-shi Higashisakuramachi,area_name_Hiroshima-ken Hiroshima-shi Kokutaijimachi,area_name_Hiroshima-ken Hiroshima-shi Nagarekawachō,area_name_Hokkaidō Abashiri-shi Minami 6 Jōhigashi,area_name_Hokkaidō Asahikawa-shi 6 Jōdōri,area_name_Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō,area_name_Hokkaidō Sapporo-shi Kita 24 Jōnishi,area_name_Hokkaidō Sapporo-shi Kotoni 2 Jō,area_name_Hokkaidō Sapporo-shi Minami 3 Jōnishi,area_name_Hyōgo-ken Amagasaki-shi Higashinanamatsuchō,area_name_Hyōgo-ken Himeji-shi Yasuda,area_name_Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,...,area_name_Tōkyō-to Kōtō-ku Kameido,area_name_Tōkyō-to Kōtō-ku Tomioka,area_name_Tōkyō-to Kōtō-ku Tōyō,area_name_Tōkyō-to Machida-shi Morino,area_name_Tōkyō-to Meguro-ku Kamimeguro,area_name_Tōkyō-to Meguro-ku Takaban,area_name_Tōkyō-to Minato-ku Akasaka,area_name_Tōkyō-to Minato-ku Shibakōen,area_name_Tōkyō-to Musashino-shi Midorichō,area_name_Tōkyō-to Nakano-ku Nakano,area_name_Tōkyō-to Nerima-ku Toyotamakita,area_name_Tōkyō-to Setagaya-ku Okusawa,area_name_Tōkyō-to Setagaya-ku Setagaya,area_name_Tōkyō-to Shibuya-ku Dōgenzaka,area_name_Tōkyō-to Shibuya-ku Shibuya,area_name_Tōkyō-to Shinagawa-ku Hiromachi,area_name_Tōkyō-to Shinjuku-ku Kabukichō,area_name_Tōkyō-to Shinjuku-ku Nishishinjuku,area_name_Tōkyō-to Suginami-ku Asagayaminami,area_name_Tōkyō-to Tachikawa-shi Izumichō,area_name_Tōkyō-to Taitō-ku Asakusa,area_name_Tōkyō-to Taitō-ku Higashiueno,area_name_Tōkyō-to Toshima-ku Mejiro,area_name_Tōkyō-to Toshima-ku Minamiikebukuro,area_name_Tōkyō-to Toshima-ku Nishiikebukuro,area_name_Tōkyō-to Ōta-ku Kamata,area_name_Ōsaka-fu Higashiōsaka-shi Aramotokita,area_name_Ōsaka-fu Ōsaka-shi Kyōmachibori,area_name_Ōsaka-fu Ōsaka-shi Kyūtarōmachi,area_name_Ōsaka-fu Ōsaka-shi Nanbasennichimae,area_name_Ōsaka-fu Ōsaka-shi Shinmachi,area_name_Ōsaka-fu Ōsaka-shi Ōgimachi,area_name_Ōsaka-fu Ōsaka-shi Ōhiraki,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,air_35512c42db0868da,2016-02-27,6,35.71778,139.56626,12,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,air_ee3a01f0c71a769f,2016-01-04,61,34.71090,137.72594,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,air_ee3a01f0c71a769f,2016-01-08,21,34.71090,137.72594,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,air_ee3a01f0c71a769f,2016-01-09,57,34.71090,137.72594,25,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,air_ee3a01f0c71a769f,2016-01-10,32,34.71090,137.72594,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

## 시각화

In [217]:
import folium

df = final_df.loc[:, ['air_store_id', 'genre', 'latitude', 'longitude']]
df = df.drop_duplicates()
df.reset_index(inplace=True)


#지도 띄우기 (지도의 중심을 지정하기 위해 위도와 경도의 평균 구하기)
map = folium.Map([df['latitude'].mean(),df['longitude'].mean()],zoom_start=7)

for i in df.index:
    sub_lat =  df.loc[i,'latitude']
    sub_long = df.loc[i,'longitude']
    
    title1 = df.loc[i, 'air_store_id']
    title2 = df.loc[i,'genre']
    #지도에 데이터 찍어서 보여주기
    folium.Marker([sub_lat,sub_long], popup = title1+'\n'+title2).add_to(map)

#한글이 안나오는 오류로 html로 trouble shooting 
map.save('map.html')
map

In [218]:
import folium
m = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=12,
    tiles='Stamen Terrain'
)

folium.Marker(
    location=[45.3288, -121.6625],
    popup='Mt. Hood Meadows',
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[45.3311, -121.7113],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[45.3300, -121.6823],
    popup='Some Other Location',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

m